In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import copy
from IPython.display import display
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [5]:
#Data Imported From Kaggle Dataset
data = pd.read_csv("data.csv")
df = data.copy()

In [6]:
#Cleaning the Artist Column
def FixArtist(artist):
    artist=artist.replace("'","").replace("'","").replace('[','').replace(']','')
    return artist

df['artists'] = df['artists'].apply(FixArtist)
data['artists'] = data['artists'].apply(FixArtist)

In [7]:
#creating dictionary for lookup after predictions
dictionary = df[["artists", "name", "key", "id","explicit"]]
#Dropping columns not suited for a Standard Scaler fit which is neccesary for NearestNeighbors
df = df.drop(['artists','id','name','release_date'], axis = 1)

#Transforming dataset to standard scaler
scaler = StandardScaler()
df_s = scaler.fit_transform(df)

In [8]:
#The NN Model
nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(df_s)

NearestNeighbors(algorithm='kd_tree', n_neighbors=10)

In [9]:
#Method for importing elsewhere
def PredictNSimilarSongs(song_name, artist, n):
    
    
    #Matches Song with the artist
    index = 0
    indices = []
    
    for i, item in enumerate(data['artists']):
        if(item == artist):
            indices.append(i)
    
    for item in indices:
        if(data['name'][item] == song_name):
            index = item
    
    #Gets Predictions with NN
    neighbor_predictions = nn.kneighbors([df_s[index]])
    
    #Finds the prediction's names and who its by in the dictionary
    list_of_predictions = neighbor_predictions[1][0].tolist()

    ten_similar_tracks_title = []
    for item in list_of_predictions:
        track_hash = dictionary['name'].iloc[item]
        explicit = dictionary['explicit'].iloc[item]
        ten_similar_tracks_title.append(track_hash)
    
    ten_similar_tracks_artists = []
    for item in list_of_predictions:
        track_hash = dictionary['artists'].iloc[item]
        ten_similar_tracks_artists.append(track_hash)

    ten_suggestions = []
    for i in range(0,n+1):
        ten_suggestions.append(ten_similar_tracks_title[i] + ' by ' + ten_similar_tracks_artists[i])
        
    
    return ten_suggestions


In [10]:
def find_word(word,df,number=10):
    df.drop_duplicates(inplace=True)
    words=df['name'].values
    artists=df['artists'].values
    t=[]
    count=0
    if word[-1]==' ':
        word=word[:-1]
    for i in words:
        if word.lower() in i.lower():
            t.append([len(word)/len(i),count])
        else:
            t.append([0,count])
        count+=1
    t.sort(reverse=True)
    s=[[words[t[i][1]],artists[t[i][1]].strip('][').split(', ')] for i in range(number)]   
    songs=[words[t[i][1]] for i in range(number)]
    artist=[artists[t[i][1]] for i in range(number)]
    x=[]
    for i in s:
        l=''
        by=''
        for j in i[1]:
            by+=j
        l+=i[0]+' by '+by
        x.append(l)
    tup=[]
    for i in range(number):
        tup.append((x[i],i))

    
    return tup,songs,artist

In [69]:
a=input('Please enter The name of the song :')
b=int(input('Please enter the number of recommendations you want: '))
tup,s,ar=find_word(a,data)
ans=widgets.Dropdown(
    options=tup,
    description='Closest Songs To '+a,
    disabled=False,
)
display(ans)

Please enter The name of the song :life
Please enter the number of recommendations you want: 9


Dropdown(description='Closest Songs To life', options=(('Life by The Walters', 0), ('LIFE by Jon Bellion', 1),…

In [70]:
prediction = PredictNSimilarSongs(s[ans.value], ar[ans.value], 9)

In [71]:
prediction

['LIFE by Saba',
 'Legend Has It by Run The Jewels',
 'Single (feat.Omeretta The Great) by Killumantii, Omeretta the Great',
 'Money by Leikeli47',
 "What's the Move (feat. Lil Uzi Vert) by Young Thug, Lil Uzi Vert",
 'Gang Shit No Lame Shit by Key Glock',
 'Like Dat by Kodak Black',
 'Shanyah by DaBaby',
 'Time for That by Kevin Gates',
 'How You Feel? (Freestyle) (feat. Lil Yachty & Ski Mask The Slump God) by DJ Scheme, Danny Towers, Lil Yachty, Ski Mask The Slump God']